### Import libraries

In [6]:
#importing libraries 
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import openpyxl
from openpyxl.drawing.image import Image as XLImage
from openpyxl import load_workbook

### Customize

In [7]:
# Customize parameters 
input_path = "0801 Raw Data.csv"
# Don’t include the extension for this file
output_path = "0801 CLV_Regression_Result"
image_path = "0801 CLV_Scatter.png"
start_date = pd.to_datetime("2024-01-01")
cutoff_date = pd.to_datetime("2024-09-30")
my_features=["ActivityPeriodDays", "TotalPurchases", "DaysSinceLastPurchase"]
my_target='CLV'

### Preprocess data

In [8]:
# Check date related issues 
df = pd.read_csv(input_path)
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], dayfirst=True, errors='coerce')

if start_date not in df["InvoiceDate"].values:
    raise ValueError("Start date not present in data.Please re-enter the correct date and try again.")
    
if cutoff_date not in df["InvoiceDate"].values:
    raise ValueError("Cutoff date not present in data.Please re-enter the correct date and try again.")
    
df.dropna(subset=["InvoiceDate", "CustomerID"], inplace=True)

clv_df = df.groupby("CustomerID").agg(
    CLV=("InvoiceAmount", "sum")
).reset_index()

df_truncated = df[(df["InvoiceDate"] >= start_date) & (df["InvoiceDate"] <= cutoff_date)]


### Extract behavior variables 

In [9]:
# Compute vaules needed to compute CLV

behavior_df = df_truncated.groupby("CustomerID").agg(
    FirstPurchaseDate=("InvoiceDate", "min"),
    LastPurchaseDate=("InvoiceDate", "max"),
    TotalPurchases=("InvoiceDate", "count")
).reset_index()

behavior_df["ActivityPeriodDays"] = (behavior_df["LastPurchaseDate"] - behavior_df["FirstPurchaseDate"]).dt.days

behavior_df["DaysSinceLastPurchase"] = (cutoff_date - behavior_df["LastPurchaseDate"]).dt.days

full_df = pd.merge(behavior_df, clv_df, on="CustomerID", how="inner")

### Run regression 

In [10]:
# Run linear regression and store model output values  

X = full_df[my_features]
y = full_df[my_target]

model = LinearRegression()
model.fit(X, y)

# Predict values
y_pred = model.predict(X)

# Build the regression equation string
equation = f"{my_target} = {intercept:.2f}"
for feature, coef in zip(my_features, coefficients):
    equation += f" + ({coef:.2f} × {feature})"
    
# Model fit
r2 = r2_score(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))
mae = mean_absolute_error(y, y_pred)

# Store model outputs
coefficients = model.coef_
intercept = model.intercept_


NameError: name 'intercept' is not defined

### Plot and save to Excel

In [ ]:
# read values to plot

full_df["Predicted_CLV"] = y_pred

plt.figure(figsize=(8, 6))
plt.scatter(full_df["CLV"], full_df["Predicted_CLV"], alpha=0.6)
plt.plot(
    [full_df["CLV"].min(), full_df["CLV"].max()],
    [full_df["CLV"].min(), full_df["CLV"].max()],
    color='red', linestyle='--'
)
plt.title("Actual vs Predicted CLV")
plt.xlabel("Actual CLV")
plt.ylabel("Predicted CLV")
plt.grid(True)
plt.tight_layout()
plt.savefig(image_path)
plt.show()
plt.close()

print("Regression Equation:")
print(equation)
print()
print("Model fit metrices")
print(f"r2 = {r2:.2f}")

print(f"RMSE = {rmse:.2f}")
print(f"MAE = {mae:.2f}")

# save data to CSV and Excel file
full_df.to_csv(f"{output_path}.csv", index=False)
full_df.to_excel(f"{output_path}_excel.xlsx", sheet_name="CLV Data", index=False)

### Create formatted  report (Optional)

In [ ]:
# Create formatted report in Excel

wb = openpyxl.load_workbook(f"{output_path}_excel.xlsx")
ws = wb.create_sheet("Regression Summary")

ws["A1"] = "Regression Equation:"
ws["A2"] = f"CLV = {intercept:.2f} + ({coefficients[0]:.2f})*ActivityPeriodDays + ({coefficients[1]:.2f})*TotalPurchases + ({coefficients[2]:.2f})*DaysSinceLastPurchase"
ws["A4"] = "Regression Statistics:"
ws["A5"] = f"R²: {r2:.4f}"
ws["A6"] = f"RMSE: {rmse:.2f}"
ws["A7"] = f"MAE: {mae:.2f}"

img = XLImage(image_path)
img.anchor = "A9"
ws.add_image(img)

ws = wb["Regression Summary"]

repeat_customers = full_df[full_df["TotalPurchases"] > 1].copy()
repeat_customers["AvgDaysBetweenPurchases"] = repeat_customers["ActivityPeriodDays"] / (repeat_customers["TotalPurchases"] - 1)

max_purchases = repeat_customers["TotalPurchases"].max()
purchase_95th = repeat_customers["TotalPurchases"].quantile(0.95)
avg_purchase_freq = repeat_customers["AvgDaysBetweenPurchases"].mean()
low_freq = repeat_customers["AvgDaysBetweenPurchases"].quantile(0.05)
high_freq = repeat_customers["AvgDaysBetweenPurchases"].quantile(0.95)

ws["N2"] = "Behavioral Insights on Repeat Customers"
ws["N3"] = f"Number of repeat customers analyzed: {len(repeat_customers)}"
ws["N4"] = f"95% made fewer than {purchase_95th:.0f} purchases (max observed: {max_purchases})"
ws["N5"] = f"Average time between purchases: {avg_purchase_freq:.1f} days"
ws["N6"] = f"Majority repurchase every {low_freq:.0f} to {high_freq:.0f} days"

# Save formatted report in a wirksheet in Excel
wb.save(f"{output_path}_excel.xlsx")
print(f"CLV Prediction has been saved at: {output_path}")
print(f"The actual and oredicyed plot has been saved at: {image_path}")